In [1]:
from pathlib import Path
from PIL import Image
import torch

from src.model import load_clip

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

#загружаем модель
model, processor = load_clip(device)

Using device: cpu


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Простой пример инференса: 

In [2]:
DATA_DIR = Path("data")

image_paths = sorted(list(DATA_DIR.glob("*.jpg")) + list(DATA_DIR.glob("*.png")))
print("Found images:")
for p in image_paths:
    print(" -", p)

image_paths

Found images:
 - data/images-2.png
 - data/istockphoto-1482199015-612x612.jpg


[PosixPath('data/images-2.png'),
 PosixPath('data/istockphoto-1482199015-612x612.jpg')]

In [3]:
texts = [
    "a magnifying glass",
    "a cute dog",
    "a smiling person",
]

texts

['a magnifying glass', 'a cute dog', 'a smiling person']

In [4]:
from torch.nn.functional import softmax

for img_path in image_paths:
    img = Image.open(img_path).convert("RGB")

    inputs = processor(
        text=texts,
        images=img,
        return_tensors="pt",
        padding=True
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    logits_per_image = outputs.logits_per_image  
    logits = logits_per_image[0]              

    probs = softmax(logits, dim=0)  # shape [num_texts]

    print(f"\nResults for {img_path.name}:")
    for text, p in zip(texts, probs):
        print(f"  {text:25s} {float(p):.3f}")



Results for images-2.png:
  a magnifying glass        1.000
  a cute dog                0.000
  a smiling person          0.000

Results for istockphoto-1482199015-612x612.jpg:
  a magnifying glass        0.000
  a cute dog                0.989
  a smiling person          0.011
